In [2]:
import sys
sys.path.append("/usr/local/lib/python3.9/site-packages")
sys.path.append("opt/anaconda3/lib/python3.8/site-packages")
import firebase_admin
from firebase_admin import firestore, credentials
import pandas as pd
import numpy as np

In [3]:
project_id = "thema3-1608728919442"
cred = credentials.Certificate("key.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [4]:
class Rate:
    def __init__(self):
        self.count = 0
        self.total = 0
    def on(self):
        self.count += 1
        self.total += 1
    def off(self):
        self.total += 1
    def show(self):
        if self.total == 0: return None
        return self.count / self.total 

In [5]:
def concat_games(games_: dict) -> list:
    games = iter(games_.values())
    outgames = [next(games)]
    for game in games:
        d_score = game[-1]["totalScore"]
        print(d_score)
        for round_ in game:
            round_["totalScore"] += d_score
        outgames += game
    print(games_, "\n\n")
    print(outgames, "\n\n\n'n")
    return outgames
            

In [6]:
def prep_game(game, id_, name):
    data = {}
        
    # coop rate
    cooprate = Rate()
    for round_ in game:
        player_action = round_["playerAction"]
        if player_action == "cooperate":
            cooprate.on()
        else: 
            cooprate.off()
        
    # dependent rates
    after_coop = Rate()
    after_def = Rate()
    for i in range(len(game)-1):

        next_round = game[i+1]
        prev_round = game[i]
        next_player_action = next_round["playerAction"]
        prev_opp_action = prev_round["opponentAction"]

        rate = after_coop if prev_opp_action == "cooperate" else after_def
        if next_player_action == "cooperate":
            rate.on()
        else:
            rate.off()

    data["after_coop_coop_rate" + name] = after_coop.show()
    data["after_defect_coop_rate" + name] = after_def.show()
    data["coop_rate" + name] = cooprate.show()
    data["id"] = id_
    data["first_round" + name] = game[0]["playerAction"]
    data["score" + name] = game[-1]["totalScore"]
        
    return data


In [7]:
from collections import defaultdict

def array_D_to_dict(d, name=""):
    return {"D11"+name: d[0], "D12"+name: d[1], 
            "D21"+name: d[2], "D22"+name: d[3]}

def normalize_arrays_in_dict(d: dict):
    for key in d.keys():
        d[key] = d[key] / d[key].sum()
    return d

def unwrap_Ds_in_dict(ds: dict):
    out = {}
    for name, d in ds.items():
        d = array_D_to_dict(d, name)
        out = {**out, **d}
    return out

def prep_all_games(games, id_):
    total_score = 0
    d = np.zeros(4)
    dpergame = defaultdict(lambda: np.zeros(4))
    coop_rate = Rate()
    for name, game in games.items():
        name = "comp" + name[:2]
        total_score += game[-1]["totalScore"]
        for round_ in game: 
            if round_["playerAction"] == "cooperate":
                coop_rate.on()
            else: 
                coop_rate.off()
            
            if round_["playerAction"] == "cooperate" and round_["opponentAction"] == "cooperate":
                d[0] += 1
                dpergame[name][0] += 1
            elif round_["playerAction"] == "defect" and round_["opponentAction"] == "cooperate":
                d[2] += 1
                dpergame[name][2] += 1
            elif round_["playerAction"] == "defect" and round_["opponentAction"] == "defect":
                d[3] += 1
                dpergame[name][3] += 1
            elif round_["playerAction"] == "cooperate" and round_["opponentAction"] == "defect":
                d[1] += 1
                dpergame[name][1] += 1
    
    d = d / d.sum()
            
    coop_rate = coop_rate.show()
    
    d = array_D_to_dict(d)
    dpergame = normalize_arrays_in_dict(dpergame)
    dpergame = unwrap_Ds_in_dict(dpergame)
    return {"final_score": total_score, 
            "coop_rate": coop_rate, 
            "id": id_, **d, **dpergame}

In [8]:
def prep_form(form, id_):
    form = {q["qId"]: q["value"] for q in form}
    form["id"] = id_
    return form

In [9]:
def prep_pqs(pqs, id_):
    pqs = {q["id"]: q["value"] for q in pqs}
    pqs["id"] = id_
    return pqs

In [10]:
users_ref = db.collection(u'responses')
docs = users_ref.stream()
forms = []
games25, games50, games75 = [], [], []
pqss = []
all_games = []
for doc in docs:
    id_ = doc.id
    doc = doc.to_dict()
    
    form = doc.get("form")
    if form: 
        form = prep_form(form, id_)
        forms.append(form)
    
    games = doc.get("games")
    if games:
        all_games.append(prep_all_games(games, id_))
        game25 = games.get("25coopgame")
        if game25:
            game25 = prep_game(game25, id_, "25")
            games25.append(game25)

        game50 = games.get("50coopgame")
        if game50:
            game50 = prep_game(game50, id_, "50")
            games50.append(game50)
        
        game75 = games.get("75coopgame")
        if game75:
            game75 = prep_game(game75, id_, "75")
            games75.append(game75)
    
    pqs = doc.get("personalQuestions")
    if pqs:
        pqs = prep_pqs(pqs, id_)
        pqss.append(pqs)
        


In [11]:
from functools import reduce
data = [pqss, games25, games50, games75, forms, all_games]
data = [pd.DataFrame(x) for x in data]
data = reduce(lambda l, r: pd.merge(l, r, on="id", how="outer"), data)

In [12]:
# calulate pre-biases/start-concentrations from game-outcomes/end-concentrations
import model
import warnings
import importlib
warnings.filterwarnings('ignore')

model = importlib.reload(model)
prebias_data = []
for _index, row in data.iterrows():
    res = model.fit_bias(row.D11, row.D12, row.D21, row.D22)
    if res.success: 
        prebias_data.append({"id":row.id, "prebias_a1":res.x[0], "prebias_b1":res.x[1]})
prebias_data = pd.DataFrame(prebias_data)

In [13]:
data = pd.merge(data, prebias_data, on="id", how="outer")

In [14]:
# calc liberalism and realism score 
pro_real = "staten_bepalen mening_vertegenwoordig staat_rationeel veiligheid1 veiligheid2".split(" ")
pro_lib = "bedrijven vrede themas binnen_buiten".split(" ")

# note all the normalizations
data["real_score"] = data[pro_real].sum(axis=1) / 5 / 7
data["lib_score"] = data[pro_lib].sum(axis=1) / 4 / 7
data["mixed_score_lower_is_lib"] = (data[pro_real].sum(axis=1) - data[pro_lib].sum(axis=1) + 3 * 7) / (4 * 7 + 3 * 7)
data["politics"] = pd.qcut(data["mixed_score_lower_is_lib"], 2, "liberalist realist".split())

data.to_csv("data.csv", sep=";")
data = data.set_index("id")
data

,leeftijd,geslacht,email,pdBekend,opmerkingen,after_coop_coop_rate25,after_defect_coop_rate25,coop_rate25,first_round25,score25,...,D11comp50,D12comp50,D21comp50,D22comp50,prebias_a1,prebias_b1,real_score,lib_score,mixed_score_lower_is_lib,politics
id,,,,,,,,,,,,,,,,,,,,,
0jd1yl8p6,19,vrouw,Beausemil04@gmail.com,ja,Ik vond het een leuke enquête! Succes met je o...,1.0,0.000000,0.4,cooperate,7.0,...,0.2,0.2,0.4,0.2,0.011364,0.559564,0.742857,0.857143,0.469388,realist
0uaijewzl,57,man,Hein.vanmiddelaar@p2.nl,ja,,1.0,1.000000,1.0,cooperate,3.0,...,0.6,0.4,0.0,0.0,NaN,NaN,0.714286,0.785714,0.489796,realist
17eq7pp6f,31,vrouw,,ja,Misschien heb ik iets gemist maar de uitleg va...,NaN,1.000000,1.0,cooperate,3.0,...,0.4,0.6,0.0,0.0,NaN,NaN,0.542857,0.750000,0.387755,realist
1cn018l7w,57,man,,nee,,0.0,0.000000,0.0,defect,13.0,...,0.0,0.0,0.6,0.4,NaN,NaN,0.514286,0.785714,0.346939,liberalist
1feix2wxq,20,man,Ikwilnietwinnen@hotmail.com,ja,Vond het leuk,1.0,0.333333,0.4,defect,7.0,...,0.2,0.2,0.2,0.4,0.396578,0.023845,0.628571,0.750000,0.448980,realist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
kj2ei8uyi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.514286,0.821429,0.326531,liberalist
momn66jov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.628571,0.785714,0.428571,realist
nbl87yduk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.485714,0.642857,0.408163,realist
